<a href="https://colab.research.google.com/github/Taikatou/economy_research/blob/master/Analytics/active_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import os
import pandas as pd
import numpy as np
import random
from statistics import mean
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
opt = tf.keras.optimizers.Adam()

In [4]:
level_cap = 5
configuration_data = {}
graph_data = { }
type_path = "D:\TestData"
game_play_data = { }
hash_data = {}
environments = {}

for filename in os.listdir(type_path):
    file_path = os.path.join(type_path, filename)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        if "level_selection" in file_path:
            if filename not in hash_data:
                hash_data[filename] = []
                environments[filename] = {}
            for i in range(len(df)):
                battle_environment = df.loc[i, 'BattleEnvironment']
                battle_id = df.loc[i, 'ID']
                if battle_environment not in environments:
                    environments[filename][battle_environment] = [0, 0, 0, 0, 0]
                if battle_id not in hash_data:
                    level = df.loc[i, 'Level']
                    environments[filename][battle_environment][level] = environments[filename][battle_environment][level] + 1
                    hash_data[filename].append(battle_id)
        elif "level_configuration" in filename:
            for i in range(len(df)):
              config = df.iloc[i].values.tolist()
              print(config)
              configuration_data[config[0]] = config[1:]
              configuration_data["1"] = config[1:]
        else:
            configurations = []
            adventurer_type = filename.split('_')[0]
            for i in range(len(df)):
                level = df.loc[i, 'Level']
                max_steps = df.loc[i, 'MaxCount']
                configuration = df.loc[i, 'Configuration']
                if configuration not in game_play_data:
                    game_play_data[configuration] = {
                        "Brawler": [[], [], [], [], []],
                        "Mage": [[], [], [], [], []],
                        "Swordsman": [[], [], [], [], []]
                    }
                    graph_data[configuration] = {}
                    configurations.append(configuration)
                
                game_play_data[configuration][adventurer_type][level-1].append(max_steps)
            
            for config_id in configurations:
                for key in game_play_data[config_id]:
                    graph_data[config_id][key] = [[], [], [], [], []]
                    for x in range(level_cap):
                        if len(game_play_data[config_id][key][x]) > 0:
                            graph_data[config_id][key][x] = mean(game_play_data[config_id][key][x])

['c8e345a5-a063-4a6c-8fd7-0e2fa8a93190', 0.4230763, 0.5729685, 0.4784689]
['d613307e-f819-4cb8-b644-061917326729', 0.4719957, 0.5025854, 0.4560405]
['c295a661-f166-4be0-b3f4-7794d4ca79b4', 0.5587001, 0.5503647, 0.5768477]
['049d5dbd-188a-4cbf-bfd3-9ff1c990a295', 0.5883144, 0.5325123, 0.4641197]
['658aba3c-1d87-4152-9e44-457568109315', 0.507669, 0.5179966, 0.4963874]
['c3e55291-d7e2-4e28-b25a-725135537939', 0.4132038, 0.4555688, 0.4205619]
['baf45b9d-fc72-441e-9858-aecfecae7b14', 0.4057138, 0.4434852, 0.4150411]
['5ddb5e77-95ed-4bff-a6cb-02e1532ac7f8', 0.5461384, 0.5928363, 0.5209548]
['63785fd0-ae64-447c-a5f3-ef134ae28b80', 0.5657998, 0.4339453, 0.4486905]
['c743d528-93e6-4471-899e-f250ff964d01', 0.4934529, 0.4983884, 0.4398314]
['4d38de33-6f15-4547-b401-85e0db1ee9d0', 0.5575743, 0.5841337, 0.4227229]
['e707cad4-3366-410c-bc05-6db274f9bdba', 0.5700304, 0.4598914, 0.5361736]
['9528eb92-7d99-476f-b988-dde40d4962d5', 0.4082832, 0.4112666, 0.4284065]
['0a3131b8-1b29-4064-8fd8-c0a06104ab28'

In [5]:
print(game_play_data)

{'d613307e-f819-4cb8-b644-061917326729': {'Brawler': [[194, 1373, 4175, 4789, 6077, 7725, 812, 2989], [2447, 8693, 9367, 555, 2900, 4659, 9057, 9048], [7205, 7874, 4763, 9837, 1579, 2726, 4356, 2323], [6952, 3472, 1604, 2376, 6714, 177], [2574, 0]], 'Mage': [[1503, 1987, 2117, 2499, 2748, 2815, 4564, 6963, 8923, 9367, 985, 178], [3390, 6724, 7725, 7974, 8211, 8211, 972, 2156, 2496, 2321], [8923, 3524, 5079, 5576, 6498, 8102, 9546, 841, 7793, 3237], [4709, 5687, 1248, 4734, 4734, 572, 2192, 2967, 8174, 2906], [8377, 0, 1458, 0]], 'Swordsman': [[194, 1202, 1503, 1505, 1563, 2004, 2004, 2748, 1893, 2368, 2989, 93, 297, 36], [1202, 1987, 2664, 2832, 3390, 3788, 446, 446, 2446, 5079, 5576, 6411], [4589, 4589, 6963, 7935, 2498, 2900, 5920, 6498, 3673, 5036, 9048, 148], [6005, 1638, 2806, 7178, 7506, 8008, 2541, 2626, 3464, 312, 585, 253], [8254, 0, 1218, 0, 1724, 0, 2790, 0]]}, '1bfdd40a-e7ae-44e5-b079-1f4b3d3cbee7': {'Brawler': [[290, 290, 1056, 1284, 3890, 9075, 2849, 4723, 6474, 514, 5682

In [6]:
# preseed reviews
reviews = {  }
from sklearn.model_selection import train_test_split
y = list(reviews.keys())
labels = []
labeled_config = []

for key in y:
    if key in graph_data:
        data = graph_data[k].values()
        labeled_config.append(data)
        labels.append(reviews[key])
        

unlabeled_data_const = configuration_data
print(unlabeled_data_const)

{'c8e345a5-a063-4a6c-8fd7-0e2fa8a93190': [0.4230763, 0.5729685, 0.4784689], '1': [0.4811949, 0.5823722, 0.5016915], 'd613307e-f819-4cb8-b644-061917326729': [0.4719957, 0.5025854, 0.4560405], 'c295a661-f166-4be0-b3f4-7794d4ca79b4': [0.5587001, 0.5503647, 0.5768477], '049d5dbd-188a-4cbf-bfd3-9ff1c990a295': [0.5883144, 0.5325123, 0.4641197], '658aba3c-1d87-4152-9e44-457568109315': [0.507669, 0.5179966, 0.4963874], 'c3e55291-d7e2-4e28-b25a-725135537939': [0.4132038, 0.4555688, 0.4205619], 'baf45b9d-fc72-441e-9858-aecfecae7b14': [0.4057138, 0.4434852, 0.4150411], '5ddb5e77-95ed-4bff-a6cb-02e1532ac7f8': [0.5461384, 0.5928363, 0.5209548], '63785fd0-ae64-447c-a5f3-ef134ae28b80': [0.5657998, 0.4339453, 0.4486905], 'c743d528-93e6-4471-899e-f250ff964d01': [0.4934529, 0.4983884, 0.4398314], '4d38de33-6f15-4547-b401-85e0db1ee9d0': [0.5575743, 0.5841337, 0.4227229], 'e707cad4-3366-410c-bc05-6db274f9bdba': [0.5700304, 0.4598914, 0.5361736], '9528eb92-7d99-476f-b988-dde40d4962d5': [0.4082832, 0.411266

In [7]:
import numpy as np

def draw_graph(game_data):
    labels = ['1', '2', '3', '4', '5', '6', '7', '8']
    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    brawler_data = game_data["Brawler"]
    rects1 = ax.bar(x - width/2, brawler_data, width, label="Brawler")

    mage_data = game_data["Mage"]
    rects2 = ax.bar(x, mage_data, width, label="Mage")

    swordsman_data = game_data["Swordsman"]
    rects3 = ax.bar(x + width/2, swordsman_data, width, label="Swordsman")

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Time Steps')
    ax.set_title('Time Steps To Next Level')
    ax.set_xticks(x, labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

In [8]:
def ask_question(adventurer_type):
  return int ( input ("On a scale of 1-5, how difficult progress is with the {adventurer_type} character compared to other playable characters\nwith 1 (unfair) too challenging, 3 fair, and 5 (unfair) too easy.") )

In [ ]:
unlabeled_gameplay_data = game_play_data.copy()
print(unlabeled_gameplay_data)
unlabeled_config_data = unlabeled_data_const.copy()
# assign random values
num_data = len(unlabeled_data_const)
num_assign = max(1, int(len(unlabeled_data_const) / 10))
label_gameplay_data = []
label_config_data = []
balance_labels = []
print("gathering " + str(num_assign) + " values")
for x in range(num_assign):
    dates = unlabeled_config_data.items()
    date, config = random.choice(list(dates))
    if date in unlabeled_gameplay_data:
        label_config_data.append(config)
        gameplay_data = unlabeled_gameplay_data[date]
        keys = ["Brawler", "Mage", "Swordsman"]
        graph_data = {}
        for key in keys:
            filter_data = np.array([row for row in gameplay_data[key] if len(row)==8])
            filter_data = filter_data.mean(axis=0)
            graph_data[key] = filter_data
        print(graph_data)
        draw_graph(graph_data)
        sword = ask_question("Swordsman")
        mage = ask_question("Mage")
        brawler = ask_question("Brawler")
        label_gameplay_data.append
        one_hot = [sword, mage, brawler]
        balance_labels.append(one_hot)
        label_gameplay_data.append(gameplay_data)
        unlabeled_gameplay_data.pop(date)
    else:
        print("Oh no")

In [18]:
print(label_data)

[[0.4719957, 0.5025854, 0.4560405], {'Brawler': [[194, 1373, 4175, 4789, 6077, 7725, 812, 2989], [2447, 8693, 9367, 555, 2900, 4659, 9057, 9048], [7205, 7874, 4763, 9837, 1579, 2726, 4356, 2323], [6952, 3472, 1604, 2376, 6714, 177], [2574, 0]], 'Mage': [[1503, 1987, 2117, 2499, 2748, 2815, 4564, 6963, 8923, 9367, 985, 178], [3390, 6724, 7725, 7974, 8211, 8211, 972, 2156, 2496, 2321], [8923, 3524, 5079, 5576, 6498, 8102, 9546, 841, 7793, 3237], [4709, 5687, 1248, 4734, 4734, 572, 2192, 2967, 8174, 2906], [8377, 0, 1458, 0]], 'Swordsman': [[194, 1202, 1503, 1505, 1563, 2004, 2004, 2748, 1893, 2368, 2989, 93, 297, 36], [1202, 1987, 2664, 2832, 3390, 3788, 446, 446, 2446, 5079, 5576, 6411], [4589, 4589, 6963, 7935, 2498, 2900, 5920, 6498, 3673, 5036, 9048, 148], [6005, 1638, 2806, 7178, 7506, 8008, 2541, 2626, 3464, 312, 585, 253], [8254, 0, 1218, 0, 1724, 0, 2790, 0]]}]


In [ ]:
train_data, train_labels, val_data, val_labels = train_test_split(label_data, labels, test_size=0.1)

In [ ]:
# Helper function for merging new history objects with older ones
def append_history(losses, val_losses, accuracy, val_accuracy, history):
    losses = losses + history.history["loss"]
    val_losses = val_losses + history.history["val_loss"]
    accuracy = accuracy + history.history["categorical_accuracy"]
    val_accuracy = val_accuracy + history.history["val_categorical_accuracy"]
    return losses, val_losses, accuracy, val_accuracy


# Plotter function
def plot_history(losses, val_losses, accuracies, val_accuracies):
    plt.plot(losses)
    plt.plot(val_losses)
    plt.legend(["train_loss", "val_loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(accuracies)
    plt.plot(val_accuracies)
    plt.legend(["train_accuracy", "val_accuracy"])
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
def make_model():
  x = Input(shape=(3,), name="Game Play Data")

  g = Dense(500, activation='relu')(x)
  g = Dense(100, activation='relu')(g)
  g = Dense(50, activation='relu')(g)
  g = Dense(3, activation='softmax')(g)
  model = Model(inputs=x, outputs=g)
  return model

In [ ]:
print(train_data)

In [ ]:
num_iterations=3
sampling_size=5000
# inspired from this https://keras.io/examples/nlp/active_learning_review_classification/
model = make_model()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=[
          keras.metrics.CategoricalAccuracy(),
          keras.metrics.FalseNegatives(),
          keras.metrics.FalsePositives(),
      ],)
losses, val_losses, accuracies, val_accuracies = [], [], [], []

# Defining checkpoints.
# The checkpoint callback is reused throughout the training since it only saves the best overall model.
checkpoint = keras.callbacks.ModelCheckpoint(
    "AL_Model.h5", save_best_only=True, verbose=1
)
# Here, patience is set to 4. This can be set higher if desired.
early_stopping = keras.callbacks.EarlyStopping(patience=4, verbose=1)

print(f"Starting to train with {len(train_data)} samples")
# Initial fit with a small subset of the training set
history = model.fit(
    x=train_data,
    y=train_labels,
    epochs=20,
    validation_data=(val_data, val_labels),
    callbacks=[checkpoint, early_stopping],
)

# Appending history
losses, val_losses, accuracies, val_accuracies = append_history(
    losses, val_losses, accuracies, val_accuracies, history
)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
used = []
label_al = labels
label_data_al = label_data
for iteration in range(num_iterations):
  # Getting predictions from previously trained model
  predictions = model.predict(unlabeled_data)
  length = min(len(predictions), 10)
  minValue = 1
  index = -1
  for x in range(length):
    if x not in used:
      predict = predictions[x]
      maxValue = max(predict)
      if maxValue < minValue:
        minValue = maxValue
        index = x
  if index != 1:
    used.append(index)
    draw_graph()
    num = ask_question("Swordsman")
    mage = ask_question("Mage")
    brawler = ask_question("Mage")
    v = int(num) - 1
    one_hot = [0, 0, 0]
    one_hot[v] = 1
    d = unlabeled_data[index]
    label_data_al.append(d)
    label_al.append(one_hot)
  print(len(label_data_al))
  print(len(label_al))
  

  al_train_data, al_val_data, al_train_labels, al_val_labels = train_test_split(label_data_al, label_al, test_size=0.1)
  print(len(al_train_data))
  print(len(al_train_labels))

  print(len(al_val_data))
  print(len(al_val_labels))

  # We recompile the model to reset the optimizer states and retrain the model
  model.compile(
      loss="categorical_crossentropy",
      optimizer="rmsprop",
      metrics=[
          keras.metrics.CategoricalAccuracy(),
          keras.metrics.FalseNegatives(),
          keras.metrics.FalsePositives(),
      ],
  )
  history = model.fit(
      x=al_train_data,
      y=al_train_labels,
      validation_data=(al_val_data, al_val_labels),
      epochs=20
  )

  # Appending the history
  losses, val_losses, accuracies, val_accuracies = append_history(
      losses, val_losses, accuracies, val_accuracies, history
  )

  # Loading the best model from this training loop
  model = keras.models.load_model("AL_Model.h5")

In [ ]:
# Plotting the overall history and evaluating the final model
  plot_history(losses, val_losses, accuracies, val_accuracies)
  print("-" * 100)
  print(
      "Test set evaluation: ",
      model.evaluate(test_dataset, verbose=0, return_dict=True),
  )
  print("-" * 100)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)